From https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24'

In [7]:
import pandas as pd

data = pd.read_csv('pedidos_pagseguro.csv', error_bad_lines=False, sep=';');
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [8]:
len(documents)

3985

In [9]:
#documents[:5]
print(documents.info)

<bound method DataFrame.info of                                           headline_text  index
0                                               bla bla      0
1                          A máquina está uma porcaria       1
2     Estou arrependido ...vcs são péssimos estou aq...      2
3     usei menos de um mês precisei de ajuda para co...      3
4       demora muito para aceitar o cartão dos clientes      4
5     Demora muito para passar a venda aprovada para...      5
6     Um sistema mais seguro e melhor suporte ao cli...      6
7     Transferir os valores para conta corrente indi...      7
8     O suporte é muito fraco, mudei meu plano para ...      8
9     A visualizacao diaria ficou muito RUIM. A VISU...      9
10              dispor de prestamos quando é nescesario     10
11    quando parcelado, depositar parceladamente, as...     11
12    comprei a minizinha chip e tinha propaganda de...     12
13    moderninha pro muito lenta, wifi fica caindo o...     13
14    Demora muito pra 

### Data Preprocessing

In [10]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

C:\Users\sgiraldo\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [11]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sgiraldo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#### Lemmatize example

In [12]:
print(WordNetLemmatizer().lemmatize('mother in law', pos='v'))

mother in law


#### Stemmer Example

In [13]:
stemmer = SnowballStemmer('english')
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data = {'original word': original_words, 'stemmed': singles})

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [14]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [16]:
doc_sample = documents[documents['index'] == 2000].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['Recebimento', 'na', 'hora', 'pelo', 'cartão', 'pre', 'pago']


 tokenized and lemmatized document: 
['recebimento', 'hora', 'pelo', 'cartão', 'pago']


In [17]:
processed_docs = documents['headline_text'].map(preprocess)

In [18]:
processed_docs[:10]

0                                                   []
1                            [máquina, está, porcaria]
2    [estou, arrependido, péssimo, estou, aqui, ten...
3    [usei, meno, precisei, ajuda, para, continuar,...
4       [demora, muito, para, aceitar, cartão, client]
5    [demora, muito, para, passar, venda, aprovada,...
6       [sistema, mai, seguro, melhor, suport, client]
7    [transferir, valor, para, conta, corrent, indi...
8    [suport, muito, fraco, mudei, plano, para, rec...
9    [visualizacao, diaria, ficou, muito, ruim, vis...
Name: headline_text, dtype: object

### Bag of words on the dataset

In [19]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [20]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 está
1 máquina
2 porcaria
3 aqui
4 arrependido
5 email
6 estou
7 falando
8 indico
9 inválido
10 merda


In [21]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [22]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4311]

IndexError: list index out of range

In [24]:
bow_doc_2000 = bow_corpus[2000]

for i in range(len(bow_doc_2000)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_2000[i][0], 
                                                     dictionary[bow_doc_2000[i][0]], 
                                                     bow_doc_2000[i][1]))

Word 18 ("cartão") appears 1 time.
Word 31 ("hora") appears 1 time.
Word 42 ("recebimento") appears 1 time.
Word 117 ("pelo") appears 1 time.
Word 127 ("pago") appears 1 time.


### TF-IDF

In [25]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [26]:
corpus_tfidf = tfidf[bow_corpus]

In [27]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[]


### Running LDA using Bag of Words

In [28]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [29]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.058*"taxa" + 0.054*"para" + 0.040*"máquina" + 0.029*"mai" + 0.027*"client" + 0.024*"muito" + 0.021*"minha" + 0.020*"juro" + 0.017*"alta" + 0.016*"outra"
Topic: 1 
Words: 0.057*"venda" + 0.050*"client" + 0.031*"muito" + 0.026*"atendimento" + 0.024*"cartão" + 0.020*"para" + 0.018*"valor" + 0.018*"taxa" + 0.016*"compra" + 0.015*"mai"
Topic: 2 
Words: 0.090*"taxa" + 0.026*"recarga" + 0.025*"pagamento" + 0.024*"celular" + 0.023*"muito" + 0.021*"problema" + 0.020*"conta" + 0.019*"venda" + 0.016*"client" + 0.016*"melhor"
Topic: 3 
Words: 0.067*"para" + 0.034*"receb" + 0.030*"minha" + 0.026*"conta" + 0.023*"venda" + 0.021*"mai" + 0.019*"taxa" + 0.019*"transferência" + 0.016*"mesmo" + 0.015*"está"
Topic: 4 
Words: 0.057*"muito" + 0.046*"para" + 0.033*"minha" + 0.033*"conta" + 0.032*"cartão" + 0.029*"taxa" + 0.023*"crédito" + 0.021*"alta" + 0.020*"bateria" + 0.018*"venda"
Topic: 5 
Words: 0.050*"cartão" + 0.036*"para" + 0.028*"máquina" + 0.025*"valor" + 0.024*"receb" + 0.022*"

Cool! Can you distinguish different topics using the words in each topic and their corresponding weights?

### Running LDA using TF-IDF

In [31]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [32]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.024*"valor" + 0.020*"mai" + 0.017*"conta" + 0.016*"para" + 0.015*"taxa" + 0.015*"cartão" + 0.014*"pagar" + 0.014*"saldo" + 0.014*"minizinha" + 0.013*"isso"
Topic: 1 Word: 0.037*"receb" + 0.032*"venda" + 0.020*"mesmo" + 0.019*"recarga" + 0.018*"taxa" + 0.017*"baixar" + 0.017*"celular" + 0.017*"para" + 0.016*"muito" + 0.015*"funciona"
Topic: 2 Word: 0.035*"taxa" + 0.028*"pagamento" + 0.023*"conta" + 0.021*"para" + 0.019*"diminuir" + 0.019*"valor" + 0.017*"transferência" + 0.013*"poder" + 0.013*"menor" + 0.013*"fazer"
Topic: 3 Word: 0.025*"crédito" + 0.023*"atendimento" + 0.019*"parcelado" + 0.018*"bateria" + 0.017*"moderninha" + 0.014*"receb" + 0.014*"débito" + 0.013*"mai" + 0.013*"facilitar" + 0.013*"para"
Topic: 4 Word: 0.059*"bateria" + 0.027*"maquina" + 0.025*"conta" + 0.024*"muito" + 0.022*"pagar" + 0.021*"melhorar" + 0.020*"máquina" + 0.016*"pelo" + 0.016*"taxa" + 0.016*"dura"
Topic: 5 Word: 0.052*"taxa" + 0.042*"alta" + 0.036*"muito" + 0.025*"problema" + 0.023*"co

### Classification of the topics

### Performance evaluation by classifying sample document using LDA Bag of Words model

In [33]:
processed_docs[2000]

['recebimento', 'hora', 'pelo', 'cartão', 'pago']

In [35]:
for index, score in sorted(lda_model[bow_corpus[2000]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.8499724268913269	 
Topic: 0.050*"cartão" + 0.036*"para" + 0.028*"máquina" + 0.025*"valor" + 0.024*"receb" + 0.022*"dinheiro" + 0.021*"hora" + 0.020*"sinal" + 0.019*"problema" + 0.018*"venda"

Score: 0.01667613908648491	 
Topic: 0.038*"para" + 0.038*"venda" + 0.035*"conta" + 0.030*"muito" + 0.020*"dinheiro" + 0.018*"maquina" + 0.018*"estou" + 0.017*"pagar" + 0.016*"pelo" + 0.015*"aplicativo"

Score: 0.016670236364006996	 
Topic: 0.040*"conta" + 0.037*"pagamento" + 0.031*"melhorar" + 0.025*"para" + 0.024*"tenho" + 0.023*"mai" + 0.022*"client" + 0.021*"venda" + 0.020*"taxa" + 0.019*"receb"

Score: 0.01666978932917118	 
Topic: 0.057*"venda" + 0.050*"client" + 0.031*"muito" + 0.026*"atendimento" + 0.024*"cartão" + 0.020*"para" + 0.018*"valor" + 0.018*"taxa" + 0.016*"compra" + 0.015*"mai"

Score: 0.01666942797601223	 
Topic: 0.071*"mai" + 0.050*"bateria" + 0.041*"maquina" + 0.036*"muito" + 0.035*"venda" + 0.026*"máquina" + 0.019*"dia" + 0.016*"conta" + 0.015*"minha" + 0.015*"valor"

Our test document has the highest probability to be part of the topic on the top.

### Performance evaluation by classifying sample document using LDA TF-IDF model

In [37]:
for index, score in sorted(lda_model_tfidf[bow_corpus[2000]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.6200292110443115	 
Topic: 0.035*"taxa" + 0.028*"pagamento" + 0.023*"conta" + 0.021*"para" + 0.019*"diminuir" + 0.019*"valor" + 0.017*"transferência" + 0.013*"poder" + 0.013*"menor" + 0.013*"fazer"

Score: 0.2466115802526474	 
Topic: 0.037*"receb" + 0.032*"venda" + 0.020*"mesmo" + 0.019*"recarga" + 0.018*"taxa" + 0.017*"baixar" + 0.017*"celular" + 0.017*"para" + 0.016*"muito" + 0.015*"funciona"

Score: 0.016672257333993912	 
Topic: 0.059*"bateria" + 0.027*"maquina" + 0.025*"conta" + 0.024*"muito" + 0.022*"pagar" + 0.021*"melhorar" + 0.020*"máquina" + 0.016*"pelo" + 0.016*"taxa" + 0.016*"dura"

Score: 0.016671443358063698	 
Topic: 0.052*"taxa" + 0.042*"alta" + 0.036*"muito" + 0.025*"problema" + 0.023*"conta" + 0.018*"para" + 0.017*"mai" + 0.016*"empréstimo" + 0.014*"cartão" + 0.013*"minha"

Score: 0.01667047291994095	 
Topic: 0.031*"para" + 0.030*"client" + 0.028*"taxa" + 0.026*"venda" + 0.020*"valor" + 0.017*"parcelamento" + 0.016*"bateria" + 0.015*"melhorar" + 0.014*"você" + 

Our test document has the highest probability to be part of the topic on the top.

### Testing model on unseen document

In [40]:
unseen_document = 'Vocês cobram muito caro'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.8199552893638611	 Topic: 0.084*"taxa" + 0.038*"conta" + 0.034*"para" + 0.029*"valor" + 0.025*"outra"
Score: 0.020010417327284813	 Topic: 0.058*"taxa" + 0.054*"para" + 0.040*"máquina" + 0.029*"mai" + 0.027*"client"
Score: 0.02000858634710312	 Topic: 0.057*"muito" + 0.046*"para" + 0.033*"minha" + 0.033*"conta" + 0.032*"cartão"
Score: 0.02000555396080017	 Topic: 0.090*"taxa" + 0.026*"recarga" + 0.025*"pagamento" + 0.024*"celular" + 0.023*"muito"
Score: 0.020004678517580032	 Topic: 0.038*"para" + 0.038*"venda" + 0.035*"conta" + 0.030*"muito" + 0.020*"dinheiro"
Score: 0.020004356279969215	 Topic: 0.071*"mai" + 0.050*"bateria" + 0.041*"maquina" + 0.036*"muito" + 0.035*"venda"
Score: 0.020003823563456535	 Topic: 0.057*"venda" + 0.050*"client" + 0.031*"muito" + 0.026*"atendimento" + 0.024*"cartão"
Score: 0.020003529265522957	 Topic: 0.040*"conta" + 0.037*"pagamento" + 0.031*"melhorar" + 0.025*"para" + 0.024*"tenho"
Score: 0.020002011209726334	 Topic: 0.050*"cartão" + 0.036*"para" + 0.